In [7]:
# Optimizado en tiempo de ejecución #

from collections import Counter
from typing import List, Tuple
import re
import json

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    usernames = []
    
    # Leer todo el archivo en memoria
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for line in lines:
        try:
            data = json.loads(line.strip())
            if 'content' in data:
                content = data['content']
                # Extraer los nombres de usuario
                usernames.extend(pattern.findall(content))
        except json.JSONDecodeError:
            continue
    
    # Contar el número de veces que aparece cada nombre de usuario
    username_counts = Counter(usernames)
    # Top 10 nombres de usuario
    top_10_usernames = username_counts.most_common(10)

    return top_10_usernames

file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json'
q3_time(file_path)


[('narendramodi', 2261),
 ('Kisanektamorcha', 1836),
 ('RakeshTikaitBKU', 1639),
 ('PMOIndia', 1422),
 ('RahulGandhi', 1125),
 ('GretaThunberg', 1046),
 ('RaviSinghKA', 1015),
 ('rihanna', 972),
 ('UNHumanRights', 962),
 ('meenaharris', 925)]